In [2]:
import os
import openai
import sys
from openai import OpenAI

openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

In [4]:
import pandas as pd
import numpy as np

In [5]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
path_chain = "/Users/teamihajlov/EELM_Application/results_qa_chain.pkl"
path_mmr = "/Users/teamihajlov/EELM_Application/results_qa_mmr.pkl"
path_compress = "/Users/teamihajlov/EELM_Application/results_qa_compress.pkl"
path_golden = "/Users/teamihajlov/EELM_Application/EELM_queries_golden.csv"
path_mini_pickle = "/Users/teamihajlov/EELM_Application/mini_pickle.pkl"

df_chain = pd.read_pickle(path_chain)
df_mmr = pd.read_pickle(path_mmr)
df_compress = pd.read_pickle(path_compress)
df_mini_pickle = pd.read_pickle(path_mini_pickle)

df_golden = pd.read_csv(path_golden)

In [7]:
#df_mini.drop(columns = 'Unnamed: 0', inplace = True)
df_mini_pickle.rename(columns={"Question": "question", "Answer": "answer", "Context": "contexts"}, inplace = True)
df_mini_pickle

,question,answer,contexts
0,What is biosphere?,The biosphere is a layer of the earth that inc...,[further cycles within them biogeochemical cyc...
1,What is fossilisation?,Fossilisation is the process where sediments f...,[sediments often on the bed of a lake or sea t...
2,"In the context of geology and rock formaiton, ...",Degradation in geology and rock formation refe...,[constituent partsdecay degradation disintegra...
3,What is metamorphism?,Metamorphism is the process where rocks underg...,[of superhot magma contact metamorphism into a...
4,What is chemical weathering?,Chemical weathering is the transformation of r...,[area which is where the reactions of chemical...


In [8]:
df_mini_pickle['contexts'][0]

['further cycles within them biogeochemical cycles represent an interesting example here charting the pathway of chemical substances such as carbon hydrogen and oxygen through both the biotic biosphere and abiotic lithosphere hydrosphere atmosphere segments of the physical world while the processes and cycles occurring on and below the lithosphere tend to move very slowly and are often imperceptible the external processes those in the atmosphere hydrosphere and biosphere have a tendency to be more rapid and',
 'biosphere have a tendency to be more rapid and more easily observed the encounter of these slower and faster processes on the earths surface contributes to the rock cycle through the continuous degradation and erosion of rocks there is a tendency to regard recycling as a modern phenomenon an attempt by the green minded to save the earth from drowning in a sea of discarded paper plastic and glass in point of fact the earth was millions of years ahead of us for the rock cycle can 

In [9]:
golden_mini = []
for gold in df_golden['Golden'][0:5]:
    golden_mini.append(gold)

In [10]:
golden_mini

['Biosphere is one of the layers of the Earth. It is a biotic segment of the Earth, meaning that includes all life forms',
 'Fossilisation is the process of parts of a dead animal or plant being turned into a part of a sediment, and thereby preserved. Normally, fossilisation preserves only the harder parts of an organism. Some kind of protective layer, often mud or sand, less frequently ice, tar, volcanic ash or tree sap, needs to build up over an organism relatively rapidly if fossilisation is to occur.',
 'Degradation the process in which a rock is being broken down as a result of disintegration. It is a part of the rock cycle.',
 'Metamorphism is the process in which rock’s chemical, mineral, and structural features change so as to adapt to a set of new conditions that usually come about as a consequence of weathering or cementation, and are different from those from which the rock originates. Heat and pressure are two important agents of metamorphism. Pressure, in fact, should be d

In [11]:
len(golden_mini)

5

In [12]:
golden_list = []
for gold in df_golden['Golden']:
    golden_list.append(gold)

print(golden_list[:5])
print(len(golden_list))

['Biosphere is one of the layers of the Earth. It is a biotic segment of the Earth, meaning that includes all life forms', 'Fossilisation is the process of parts of a dead animal or plant being turned into a part of a sediment, and thereby preserved. Normally, fossilisation preserves only the harder parts of an organism. Some kind of protective layer, often mud or sand, less frequently ice, tar, volcanic ash or tree sap, needs to build up over an organism relatively rapidly if fossilisation is to occur.', 'Degradation the process in which a rock is being broken down as a result of disintegration. It is a part of the rock cycle.', 'Metamorphism is the process in which rock’s chemical, mineral, and structural features change so as to adapt to a set of new conditions that usually come about as a consequence of weathering or cementation, and are different from those from which the rock originates. Heat and pressure are two important agents of metamorphism. Pressure, in fact, should be divi

In [13]:
df_chain['ground_truth'] = golden_list
df_mmr['ground_truth'] = golden_list
df_compress['ground_truth'] = golden_list

In [14]:
df_chain.rename(columns={"Question": "question", "Answer": "answer", "Context": "contexts"}, inplace = True)
df_mmr.rename(columns={"Question": "question", "Answer": "answer", "Context": "contexts"}, inplace = True)
df_compress.rename(columns={"Question": "question", "Answer": "answer", "Context": "contexts"}, inplace = True)

In [15]:
df_chain.head()

,question,answer,contexts,ground_truth
0,What is biosphere?,The biosphere is a layer of the earth that inc...,[further cycles within them biogeochemical cyc...,Biosphere is one of the layers of the Earth. I...
1,What is fossilisation?,Fossilisation is the process where sediments f...,[sediments often on the bed of a lake or sea t...,Fossilisation is the process of parts of a dea...
2,"In the context of geology and rock formaiton, ...",Degradation in geology refers to the process o...,[constituent partsdecay degradation disintegra...,Degradation the process in which a rock is bei...
3,What is metamorphism?,Metamorphism is the process where rocks underg...,[of superhot magma contact metamorphism into a...,Metamorphism is the process in which rock’s ch...
4,What is chemical weathering?,Chemical weathering is the transformation of r...,[area which is where the reactions of chemical...,Chemical weathering implies transformation of ...


In [16]:
df_mmr.head()

,question,answer,contexts,ground_truth
0,What is biosphere?,The biosphere is a layer of the earth that inc...,[further cycles within them biogeochemical cyc...,Biosphere is one of the layers of the Earth. I...
1,What is fossilisation?,Fossilisation is the process of parts of a dea...,[sediments often on the bed of a lake or sea t...,Fossilisation is the process of parts of a dea...
2,"In the context of geology and rock formaiton, ...",Degradation in geology refers to the process o...,[constituent partsdecay degradation disintegra...,Degradation the process in which a rock is bei...
3,What is metamorphism?,Metamorphism is the process in which the chemi...,[of superhot magma contact metamorphism into a...,Metamorphism is the process in which rock’s ch...
4,What is chemical weathering?,Chemical weathering occurs when the chemical s...,[area which is where the reactions of chemical...,Chemical weathering implies transformation of ...


In [17]:
df_compress.head()

,question,answer,contexts,ground_truth
0,What is biosphere?,The biosphere is the layer of the earth that i...,"[biotic biosphere, biosphere a layer of the ea...",Biosphere is one of the layers of the Earth. I...
1,What is fossilisation?,Fossilisation is the process of preserving the...,[sediments often on the bed of a lake or sea t...,Fossilisation is the process of parts of a dea...
2,"In the context of geology and rock formaiton, ...",Degradation in geology refers to the process o...,[degradation the process in which a rock is be...,Degradation the process in which a rock is bei...
3,What is metamorphism?,Metamorphism is the process where minerals rec...,[trigger metamorphic changes through thermic a...,Metamorphism is the process in which rock’s ch...
4,What is chemical weathering?,Chemical weathering is the process where rocks...,[chemical weathering occurs when the chemical ...,Chemical weathering implies transformation of ...


In [18]:
print(df_chain.shape)
print(df_mmr.shape)
print(df_compress.shape)

(49, 4)
(49, 4)
(49, 4)


In [19]:
type(df_compress['contexts'][0])

list

## Generating embeddings

In [20]:
#def get_embedding(text):
#embedding = OpenAIEmbeddings()

## Evaluation

In [21]:
from langchain.evaluation import load_evaluator

evaluation_llm = ChatOpenAI(model="gpt-4")
embeddings = OpenAIEmbeddings()

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## Preparing data and evaluation function

In [22]:
#!pip install evaluate

In [23]:
def get_bleu_variables(df, colname):
    predictions = []
    for answer in df[colname].tolist():
        answers = []
        answers.append(answer)
        predictions.append(answers)
    return predictions

In [24]:
predictions_chain = get_bleu_variables(df_chain, 'answer')
print(predictions_chain[2])
print(len(predictions_chain))
print(type(predictions_chain[2]))

['Degradation in geology refers to the process of a rock being broken down as a result of disintegration. This process involves the transformation of large deposits into smaller pieces of rock and minerals, eventually leading to the formation of soils. Weathering and erosion are the two interlinked processes that contribute to degradation in rock formation.']
49
<class 'list'>


In [25]:
predictions_mmr = get_bleu_variables(df_mmr, 'answer')
print(predictions_mmr[2])
print(len(predictions_mmr))
print(type(predictions_mmr[2]))

["Degradation in geology refers to the process of a rock being broken down through disintegration and decay. This process contributes to the continuous erosion and degradation of rocks in the rock cycle. The encounter of slower and faster processes on Earth's surface leads to the degradation and erosion of rocks."]
49
<class 'list'>


In [26]:
predictions_compress = get_bleu_variables(df_compress, 'answer')
print(predictions_compress[2])
print(len(predictions_compress))
print(type(predictions_compress[2]))

['Degradation in geology refers to the process of rocks being broken down through disintegration, weathering, and erosion. It involves the gradual breakdown of rocks into smaller particles over time.']
49
<class 'list'>


In [27]:
references = get_bleu_variables(df_chain, 'ground_truth')
print(references[2])
print(len(references))
print(type(references[2]))

['Degradation the process in which a rock is being broken down as a result of disintegration. It is a part of the rock cycle.']
49
<class 'list'>


In [28]:
predictions_chain_mini = predictions_chain[0:5]
references_mini = references[0:5]

In [29]:
#!pip install sacrebleu

In [30]:
def get_eval(score, predictions, references):
    results_final = []
    bleu_results = []
    for prediction, reference in zip(predictions, references):
        results = score.compute(predictions = prediction, references = reference)
        bleu_results.append(results)
        
    results_final.append(bleu_results)
    
    for r in results_final:
        df = pd.DataFrame(r)
    
    return df

In [31]:
import evaluate

bleu = evaluate.load("bleu")

bleu_mini = get_eval(bleu, predictions_chain_mini, references_mini)

In [32]:
bleu_mini

,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length
0,0.140787,"[0.36363636363636365, 0.23255813953488372, 0.0...",1.000000,1.760000,44,25
1,0.040756,"[0.3404255319148936, 0.08695652173913043, 0.04...",0.551152,0.626667,47,75
2,0.164405,"[0.3220338983050847, 0.1724137931034483, 0.122...",1.000000,2.269231,59,26
3,0.030302,"[0.4716981132075472, 0.11538461538461539, 0.03...",0.374886,0.504762,53,105
4,0.489862,"[0.8333333333333334, 0.7021276595744681, 0.652...",0.701758,0.738462,48,65


In [33]:
from rouge_score import rouge_scorer, scoring
import evaluate

rouge = evaluate.load('rouge')

rouge_mini = get_eval(rouge, predictions_chain_mini, references_mini)

In [34]:
rouge_mini

,rouge1,rouge2,rougeL,rougeLsum
0,0.531250,0.354839,0.343750,0.343750
1,0.224299,0.076190,0.186916,0.186916
2,0.430380,0.233766,0.405063,0.405063
3,0.281690,0.071429,0.183099,0.183099
4,0.723810,0.621359,0.685714,0.685714


## BLEU

In [35]:
import evaluate

bleu = evaluate.load("bleu")

In [36]:
bleu_chain = get_eval(bleu, predictions_chain, references)

bleu_chain.head()

,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length
0,0.140787,"[0.36363636363636365, 0.23255813953488372, 0.0...",1.000000,1.760000,44,25
1,0.040756,"[0.3404255319148936, 0.08695652173913043, 0.04...",0.551152,0.626667,47,75
2,0.164405,"[0.3220338983050847, 0.1724137931034483, 0.122...",1.000000,2.269231,59,26
3,0.030302,"[0.4716981132075472, 0.11538461538461539, 0.03...",0.374886,0.504762,53,105
4,0.489862,"[0.8333333333333334, 0.7021276595744681, 0.652...",0.701758,0.738462,48,65


In [37]:
bleu_mmr = get_eval(bleu, predictions_mmr, references)

bleu_mmr.head()

,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length
0,0.129873,"[0.325, 0.20512820512820512, 0.078947368421052...",1.000000,1.600000,40,25
1,0.336317,"[0.6166666666666667, 0.423728813559322, 0.3793...",0.778801,0.800000,60,75
2,0.084423,"[0.32075471698113206, 0.1346153846153846, 0.05...",1.000000,2.038462,53,26
3,0.058295,"[0.5686274509803921, 0.22, 0.10204081632653061...",0.346864,0.485714,51,105
4,0.340445,"[0.6071428571428571, 0.41818181818181815, 0.33...",0.851535,0.861538,56,65


In [38]:
bleu_compress = get_eval(bleu, predictions_compress, references)

bleu_compress.head()

,bleu,precisions,brevity_penalty,length_ratio,translation_length,reference_length
0,0.152366,"[0.7142857142857143, 0.38461538461538464, 0.25...",0.455794,0.560000,14,25
1,0.096309,"[0.40476190476190477, 0.21951219512195122, 0.1...",0.455794,0.560000,42,75
2,0.000000,"[0.42424242424242425, 0.125, 0.032258064516129...",1.000000,1.269231,33,26
3,0.025234,"[0.4117647058823529, 0.08, 0.04081632653061224...",0.346864,0.485714,51,105
4,0.280996,"[0.7755102040816326, 0.4166666666666667, 0.297...",0.721422,0.753846,49,65


In [39]:
bleu_chain.to_pickle('bleu_chain.pkl')
bleu_mmr.to_pickle('bleu_mmr.pkl')
bleu_compress.to_pickle('bleu_compress.pkl')

## ROUGE

In [40]:
#!pip install rouge_score

In [41]:
from rouge_score import rouge_scorer, scoring
import evaluate

rouge = evaluate.load('rouge')

In [42]:
rouge_chain = get_eval(rouge, predictions_chain, references)

rouge_chain.head()

,rouge1,rouge2,rougeL,rougeLsum
0,0.531250,0.354839,0.343750,0.343750
1,0.224299,0.076190,0.186916,0.186916
2,0.430380,0.233766,0.405063,0.405063
3,0.281690,0.071429,0.183099,0.183099
4,0.723810,0.621359,0.685714,0.685714


In [45]:
rouge.compute(predictions=predictions_chain[0],
                        references=references[0],
                        use_aggregator=True)

{'rouge1': 0.53125,
 'rouge2': 0.3548387096774194,
 'rougeL': 0.34375000000000006,
 'rougeLsum': 0.34375000000000006}

In [44]:
rouge_mmr= get_eval(rouge, predictions_mmr, references)

rouge_mmr.head()

,rouge1,rouge2,rougeL,rougeLsum
0,0.508475,0.280702,0.372881,0.372881
1,0.545455,0.420168,0.495868,0.495868
2,0.400000,0.164384,0.346667,0.346667
3,0.347826,0.132353,0.246377,0.246377
4,0.563636,0.407407,0.490909,0.490909


In [186]:
rouge_compress= get_eval(rouge, predictions_compress, references)

rouge_compress.head()

,rouge1,rouge2,rougeL,rougeLsum
0,0.666667,0.411765,0.611111,0.611111
1,0.285714,0.174757,0.247619,0.247619
2,0.452830,0.117647,0.339623,0.339623
3,0.241135,0.057554,0.156028,0.156028
4,0.672897,0.380952,0.429907,0.429907


In [187]:
rouge_chain.to_pickle('rouge_chain.pkl')
rouge_mmr.to_pickle('rouge_mmr.pkl')
rouge_compress.to_pickle('rouge_compress.pkl')

## RAGAS

In [23]:
#!pip install ragas

In [24]:
dict_chain = {'question' : df_chain['question'].tolist(),
             'answer' : df_chain['answer'].tolist(),
             'contexts' : df_chain['contexts'].tolist(),
             'ground_truth' : df_chain['ground_truth'].tolist(),}

In [25]:
dict_mmr = {'question' : df_mmr['question'].tolist(),
             'answer' : df_mmr['answer'].tolist(),
             'contexts' : df_mmr['contexts'].tolist(),
             'ground_truth' : df_mmr['ground_truth'].tolist(),}

In [26]:
dict_compress = {'question' : df_compress['question'].tolist(),
             'answer' : df_compress['answer'].tolist(),
             'contexts' : df_compress['contexts'].tolist(),
             'ground_truth' : df_compress['ground_truth'].tolist(),}

In [27]:
from datasets import Dataset

dataset_chain = Dataset.from_dict(dict_chain)
dataset_mmr = Dataset.from_dict(dict_mmr)
dataset_compress = Dataset.from_dict(dict_compress)

In [31]:
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    answer_relevancy,
    faithfulness,
    context_recall,
    answer_correctness
)

# list of metrics we're going to use
metrics = [
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
]

In [35]:
score_chain = evaluate(dataset_chain, metrics = metrics, raise_exceptions=False)
score_chain.to_pandas()

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Evaluating:   0%|          | 0/245 [00:00<?, ?it/s]

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascor

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascor

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascor

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascor

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascor

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascor

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascor

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,answer_correctness,context_recall,context_precision
0,What is biosphere?,The biosphere is a layer of the earth that inc...,[further cycles within them biogeochemical cyc...,Biosphere is one of the layers of the Earth. I...,NaN,NaN,NaN,NaN,NaN
1,What is fossilisation?,Fossilisation is the process where sediments f...,[sediments often on the bed of a lake or sea t...,Fossilisation is the process of parts of a dea...,NaN,NaN,NaN,NaN,NaN
2,"In the context of geology and rock formaiton, ...",Degradation in geology refers to the process o...,[constituent partsdecay degradation disintegra...,Degradation the process in which a rock is bei...,NaN,NaN,NaN,1.000000,0.991503
3,What is metamorphism?,Metamorphism is the process where rocks underg...,[of superhot magma contact metamorphism into a...,Metamorphism is the process in which rock’s ch...,0.660353,0.833333,1.000000,1.000000,0.736879
4,What is chemical weathering?,Chemical weathering is the transformation of r...,[area which is where the reactions of chemical...,Chemical weathering implies transformation of ...,1.000000,0.887500,0.943937,0.878734,0.887500
5,What is sporopollenin?,Sporopollenin is a biological polymer found in...,[is constructed of sporopollenin a substance w...,Sporopollenin is a substance that is both stro...,1.000000,1.000000,0.685957,1.000000,1.000000
6,What is detritus?,Detritus is pieces of waste material left afte...,[animals characterised by two pairs of antenna...,Detritus are pieces of waste material left aft...,1.000000,1.000000,0.746577,1.000000,1.000000
7,What is topography?,Topography refers to the configuration of a la...,[to soil fertility as mentioned above the acti...,Topography is a configuration of a land surfac...,1.000000,0.980301,0.908397,1.000000,1.000000
8,What is beryl?,Beryl is a mineral made up of a silicate of be...,[a planetbelieve an action of considering some...,Beryl is a family of minerals made up of a sil...,1.000000,0.732826,1.000000,0.500000,1.000000
9,What is a metamorphic rock?,A metamorphic rock is formed when existing roc...,[as layers of sediment are laid down and becom...,Metamorphic rock is a type of rock that occurs...,0.956104,0.876697,1.000000,0.500000,1.000000


In [39]:
score_chain = score_chain.to_pandas()

In [48]:
score_chain.to_pickle('score_chain.pkl')

In [49]:
score_chain.to_csv('score_chain.csv')

In [45]:
score_chain.isnull().sum()

question              0
answer                0
contexts              0
ground_truth          0
faithfulness          3
answer_relevancy      3
answer_correctness    3
context_recall        2
context_precision     2
dtype: int64

In [50]:
score_mmr = evaluate(dataset_mmr, metrics = metrics, raise_exceptions=False)

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Evaluating:   0%|          | 0/245 [00:00<?, ?it/s]

/Users/teamihajlov/anaconda3/lib/python3.11/json/decoder.py:353: RuntimeWarning: coroutine 'Executor.wrap_callable_with_index.<locals>.wrapped_callable_async' was never awaited
  obj, end = self.scan_once(s, idx)
/Users/teamihajlov/anaconda3/lib/python3.11/json/decoder.py:353: RuntimeWarning: coroutine 'Executor.wrap_callable_with_index.<locals>.wrapped_callable_async' was never awaited
  obj, end = self.scan_once(s, idx)
/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/base_events.py:692: ResourceWarning: unclosed event loop <_UnixSelectorEventLoop running=False closed=False debug=False>
  _warn(f"unclosed event loop {self!r}", ResourceWarning, source=self)
Task was destroyed but it is pending!
task: <Task pending name='Task-30' coro=<as_completed.<locals>.sema_coro() running at /Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py:38> wait_for=<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/teamihajlov/anaconda3/lib/python3.11/asynci

Task exception was never retrieved
future: <Task finished name='Task-62' coro=<as_completed.<locals>.sema_coro() done, defined at /Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py:36> exception=AssertionError('LLM must be set')>
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascore
    score = await self._ascore(row=row, callbacks=group_cm, is_async=is_async)
            ^^^^^^^^^^^

Task was destroyed but it is pending!
task: <Task pending name='Task-80' coro=<as_completed.<locals>.sema_coro() running at /Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py:37> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[as_completed.<locals>._on_completion() at /Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py:592]>
Task was destroyed but it is pending!
task: <Task pending name='Task-81' coro=<as_completed.<locals>.sema_coro() running at /Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py:37> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[as_completed.<locals>._on_completion() at /Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py:592]>
Task was destroyed but it is pending!
task: <Task pending name='Task-82' coro=<as_completed.<locals>.sema_coro() running at /Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py:37> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[as_compl

...

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascor

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascor

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascor

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascor

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascor

In [51]:
score_mmr_df = score_mmr.to_pandas()

In [56]:
score_mmr_df.to_pickle('score_mmr_df.pkl')

In [57]:
score_mmr_df.to_csv('score_mmr_df.csv')

In [53]:
score_chain.isnull().sum()

question              0
answer                0
contexts              0
ground_truth          0
faithfulness          3
answer_relevancy      3
answer_correctness    3
context_recall        2
context_precision     2
dtype: int64

In [54]:
score_compress = evaluate(dataset_compress, metrics = metrics, raise_exceptions=False)

/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Evaluating:   0%|          | 0/245 [00:00<?, ?it/s]

Runner in Executor raised an exception
Traceback (most recent call last):
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 38, in sema_coro
    return await coro
           ^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 93, in ascore
    raise e
  File "/Users/teamihajlov/anaconda3/lib/python3.11/site-packages/ragas/metrics/base.py", line 89, in ascor

In [55]:
score_compress_df = score_compress.to_pandas()

In [61]:
score_compress_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,answer_correctness,context_recall,context_precision
0,What is biosphere?,The biosphere is the layer of the earth that i...,"[biotic biosphere, biosphere a layer of the ea...",Biosphere is one of the layers of the Earth. I...,NaN,NaN,1.000000,0.946266,0.614154
1,What is fossilisation?,Fossilisation is the process of preserving the...,[sediments often on the bed of a lake or sea t...,Fossilisation is the process of parts of a dea...,1.000000,0.500000,1.000000,0.999999,0.489517
2,"In the context of geology and rock formaiton, ...",Degradation in geology refers to the process o...,[degradation the process in which a rock is be...,Degradation the process in which a rock is bei...,1.000000,1.000000,0.944470,0.837568,1.000000
3,What is metamorphism?,Metamorphism is the process where minerals rec...,[trigger metamorphic changes through thermic a...,Metamorphism is the process in which rock’s ch...,1.000000,1.000000,1.000000,0.684073,1.000000
4,What is chemical weathering?,Chemical weathering is the process where rocks...,[chemical weathering occurs when the chemical ...,Chemical weathering implies transformation of ...,1.000000,1.000000,1.000000,0.745603,1.000000
5,What is sporopollenin?,Sporopollenin is a strong and chemically stabl...,[is constructed of sporopollenin a substance w...,Sporopollenin is a substance that is both stro...,1.000000,1.000000,1.000000,0.809257,1.000000
6,What is detritus?,Detritus is pieces of waste material left afte...,[debris a collection of fragments of rock\ndeb...,Detritus are pieces of waste material left aft...,1.000000,1.000000,1.000000,0.535642,1.000000
7,What is topography?,Topography refers to the configuration of a la...,[topography and slope the shape of the landsca...,Topography is a configuration of a land surfac...,0.500000,1.000000,0.940690,0.610805,1.000000
8,What is beryl?,Beryl is a mineral composed of a silicate of b...,[beryl a mineral made up of a silicate of bery...,Beryl is a family of minerals made up of a sil...,1.000000,1.000000,1.000000,0.455568,0.250000
9,What is a metamorphic rock?,A metamorphic rock is formed when existing roc...,[marble and slate are examples of metamorphic ...,Metamorphic rock is a type of rock that occurs...,1.000000,1.000000,0.927397,0.640727,1.000000


In [60]:
score_compress_df.isnull().sum()

question              0
answer                0
contexts              0
ground_truth          0
faithfulness          3
answer_relevancy      1
answer_correctness    0
context_recall        0
context_precision     0
dtype: int64

In [58]:
score_compress_df.to_pickle('score_compress.pkl')

In [59]:
score_compress_df.to_csv('score_compress.csv')